# Lung Cancer Post-Translational Modification and Gene Expression Regulation
Lung cancer is a complex disease that is known to be regulated at the post-translational modification level, e.g. phosphorylation driven by kinases. Our collaborators at [Cell Signaling Incorporated](https://www.cellsignal.com/) used Tandem Mass Tagg (TMT) mass spectrometry to measure differential phosphorylation, acetylation, and methylation in a panel of 42 lung cancer cell lines compared to non-cancerous lung tissue. Gene expression data from 37 of these lung cancer cell lines was also independently obtained from the publically available Cancer Cell Line Encyclopedia [(CCLE)](https://portals.broadinstitute.org/ccle/home). 

This post-translational modification (PTM) and gene expression data was pre-processed (normalized, filtered, etc) in the [CST_Data_Processing.ipynb](http://nbviewer.jupyter.org/github/MaayanLab/CST_Lung_Cancer_Viz/blob/master/notebooks/CST_Data_Processing.ipynb) notebook. This notebook will visualize PTM, Expression, and merged PTM-Expression datasets and identify clusters of PTMs/genes for further analysis. 

### Load Data and Clustergrammer-Widget
First, we will make an instance of the [Clustergrammer-PY Network](http://clustergrammer.readthedocs.io/clustergrammer_py.html#clustergrammer-py-api) class that will be used to load, analyze, and visualize our data. For more information see [Clustergrammer-PY API](http://clustergrammer.readthedocs.io/clustergrammer_py.html#clustergrammer-py-api) and [Clustergrammer-Widget](http://clustergrammer.readthedocs.io/clustergrammer_widget.html).

In [2]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

net.load_file('../lung_cellline_3_1_16/lung_cl_all_ptm/precalc_processed/CST_CCLE_ptm.txt')

print(net.dat['mat'].shape)

(1730, 37)


# CST Post Translational Modification Lung Cancer Data
Here we will visualize pre-processed PTM data from our collaborators at CST (see [CST_Data_Processing.ipynb](http://nbviewer.jupyter.org/github/MaayanLab/CST_Lung_Cancer_Viz/blob/master/notebooks/CST_Data_Processing.ipynb) for information on data processing). In this dataset, 37 lung cancer cell lines had differential phosphorylation, methylation, and acetylation measured relative to non-cancerous lung tissue. PTM levels were quantile normalized in each cell, PTMs with more than 7 missing values were disguarded, and PTM levels across cell lines wre Z-score normalized to highlight differential regulation across lung cancer cell lines.

In [3]:
# manually color PTM type and cell line histology
net.set_cat_color('row', 1, 'Data-Type: phospho', 'red')
net.set_cat_color('row', 1, 'Data-Type: Rme1', 'purple')
net.set_cat_color('row', 1, 'Data-Type: AcK', 'blue')
net.set_cat_color('row', 1, 'Data-Type: Kme1', 'grey')
net.set_cat_color('col', 1, 'Histology: SCLC', 'red')
net.set_cat_color('col', 1, 'Histology: NSCLC', 'blue')

net.cluster(views=[])
net.widget()

### Cell Lines Cluster According to Histology and Mutation
We can see that cell lines cluster according to their histology - almost all NSCLC cell lines (blue column category) cluster together (except for H2106) and almost all SCLC cell lines (red column category) cluster together). We can see two high-level clusters of PTMs that have either high/low levels in SCLC cell lines and low/high levels, respectively in NSCLC cell lines. The cluster with high levels in SCLC cell lines mainly composed of phosphorylation, arginine methylation, and lysine acetylation, while the cluster with low levels in SCLC cell lines is almost entierly composed of phosphorylation (red row category). Below, we will use the [interactive dendrogram](http://clustergrammer.readthedocs.io/interacting_with_viz.html#interactive-dendrogram) in combination with the ``widget_df`` method (see [Clustergrammer-PY API](http://clustergrammer.readthedocs.io/clustergrammer_py.html#clustergrammer-py-api)) to export these clusters to TSV for further analysis. 

We also included known mutations in lung cancer cell lines as additional column categories (e.g. mut-TP53). We see that cell lines appear to cluster according to common mutations in: EGFR, KRAS, and RB1. Mutations in these genes may be the drivers behind their common PTM regulation. 

### PTMs cluster according to Type
We see that PTMs (rows) also tend to cluster according to their type (e.g. phosphorylation). However, we also see that different types of modifications co-cluster. 

In [4]:
# # here we are using the interactive dendrogram (not shown) to select clusters and export them to TSVs using
# # the widget_df method. 
# ptm_sclc = net.widget_df()
# ptm_nsclc = net.widget_df()
# ptm_sclc.to_csv('histology_clusters/ptm_sclc.txt', sep='\t')
# ptm_nsclc.to_csv('histology_clusters/ptm_nsclc.txt', sep='\t')

# Gene Expression Data
We obtained gene expression data from the CCLE for 37 lung cancer cell lines assayed by our collaborators at CST. This independent dataset can be used to find novel correlations between differentially expressed genes and PTMs as well as determine whether lung cancer cell lines behave similarly in gene expression space and PTM space. Gene expression data for the 37 cell lines obtained from the CCLE dataset, 1000 genes with the greatest variance across the cell lines were kept, and genes were Z-score normalized across all cell lines to highlight differential expression across lung cancer cell lines. 

In [5]:
net.load_file('../lung_cellline_3_1_16/lung_cl_all_ptm/precalc_processed/CST_CCLE_Exp.txt')
net.set_cat_color('row', 1, 'Data-Type: Exp', 'yellow')
net.cluster(views=[])
net.widget()

### Cell Lines Cluster According to Histology and Mutation
Similarly to what we saw with the PTM data above, we see that cell lines cluster according to their histology and shared mutations. Again, we see that the cell line H2106 is the only NSCLC cell line that clusters with the SCLC cell lines. We also see that cell lines cluster based on the RB1 mutation, but less so on KRAS and EGFR mutations. Cell lines may cluster based on EGFR mutation in PTM space and not gene-expression space since EGFR is a kinase that effects phosphorylation levels in PTM space. 

We also see two high-level clusters of genes that have high/low expression in SCLC cell lines and vice versa in NSCLC cell lines.

### Enrichment Analysis

In [5]:
# exp_sclc = net.widget_df()
# exp_nsclc = net.widget_df()
# exp_sclc.to_csv('histology_clusters/exp_sclc.txt', sep='\t')
# exp_nsclc.to_csv('histology_clusters/exp_nsclc.txt', sep='\t')

We also have expression data for these cell lines and we can see that the cell lines also cluster according to their histology based on this data. 

# Merge PTM and Gene Expression Data
We can merge the two independent data types together to idenfity cell line clusters and measurement clusters that cross data-type.

In [6]:
net.load_file('../lung_cellline_3_1_16/lung_cl_all_ptm/precalc_processed/CST_CCLE_merge.txt')
net.cluster(views=[])
net.widget()

Similarly to the above two heatmaps with PTM and expression data our merged PTM-expression data heatmap shows: cell lines cluster according to their histology, and we have two broad clusters of up- and down-regulated PTMs/genes in SCLC and NSCLC cell ilnes. 

In [7]:
# merge_sclc = net.widget_df()
# merge_nsclc = net.widget_df()
# merge_sclc.to_csv('histology_clusters/merge_sclc.txt', sep='\t')
# merge_nsclc.to_csv('histology_clusters/merge_nsclc.txt', sep='\t')